# I . Strategy Pattern 
### _In Python_

## I.1 The OOP implementation

The Strategy Pattern defines a **family of algorithms**, encapsulates each one of them and makes them interchangeable. This pattern favours the modificacion of these algorithms without affecting the clients using it.

- The encapsulation is what makes the algorithms interchangeable.
- i.e., this pattern de-couples the algorithms from the object (client) using them. So, changing the algorithms does not affect the client's code


Why use it?

1. Inheritance is not intended for code reuse. i.e., strategy pattern minimizes inheritance.
2. The strategy pattern avoids the hard coding of behaviours into the intended overall classes. It favours the creation of abstract behaviours (by encapsulating each algorithm in its own class), which are latter injected into the class using them.

#### Example: List classifier
*(This is the most 'straight forward implementation from the books in Python, using the abc library to create the abstract classes)*

In [5]:
from abc import ABC, abstractmethod
import random

#We create the abstract class for the classification method
class I_NumClassifier(ABC):
    @abstractmethod
    def classify(self,*args,**kwargs):
        pass

#We encapsulate an specific behaviour (algorithm or concrete behaviour) in its own class
class RandomClassifier(I_NumClassifier):
    def classify(self,lista:list) -> list:
        lista2 = lista.copy()
        random.shuffle(lista2)
        return lista2

#We create the client that uses the classification algorithm
class List_reader:
    def __init__(self,lista:list) -> None:
        self.lista = lista
    
    def classify_list(self,strategy: I_NumClassifier):
        print(f'The provided classifier is {strategy().__class__.__name__}')
        print(f'Given list {self.lista}')
        sorted_list = strategy().classify(self.lista)
        print(f'Classified list {sorted_list}')

Let us try some examples now

In [7]:
lista = [1,4,7,0,12,5,43]
reader = List_reader(lista)
reader.classify_list(RandomClassifier)


The provided classifier is RandomClassifier
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list [0, 7, 12, 4, 1, 43, 5]


We can always create other concrete behaviours and inject them into the client, without changing the client's code

In [8]:
class ReversedClassifier(I_NumClassifier):
    def classify(self,lista:list) -> list:
        lista2 = lista.copy()
        lista2.reverse()
        return lista2

class SortingClassifier(I_NumClassifier):
    def classify(self,lista:list) -> list:
        lista2 = lista.copy()
        lista2.sort()
        return lista2

class BlackHoleClassifier(I_NumClassifier):
    def classify(self,lista:list) -> list:
        return []

Let us try them

In [9]:
reader.classify_list(ReversedClassifier)
reader.classify_list(SortingClassifier)
reader.classify_list(BlackHoleClassifier)



The provided classifier is ReversedClassifier
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list [43, 5, 12, 0, 7, 4, 1]
The provided classifier is SortingClassifier
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list [0, 1, 4, 5, 7, 12, 43]
The provided classifier is BlackHoleClassifier
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list []


Notice how we didn't modify the List_reader class at all, and we didn't even required a re-instantiation of the class. It works perfectly well by calling different strategies from the execute() method

## I.2 Functional Approach to the Strategy Pattern

Given that Python provides a good basis for functional programming, let usw try this option.
Instead of creating classes for the concrete strategies, we can create functions that are later directly injected into the client

In [4]:
def reversedClassifier_function(lista : list) -> list:
    lista2 = lista.copy()
    lista2.reverse()
    return lista2

def blackHoleClassifier_function(lista : list) -> list:
    return []

def randomClassifier_function(lista : list) -> list:
    lista2 = lista.copy()
    random.shuffle(lista2)
    return lista2

def sortingClassifier_function(lista:list) -> list:
    lista2 = lista.copy()
    lista2.sort()
    return lista2


#We also have to modify the client to accept functions

class List_reader_functional:
    def __init__(self,lista:list) -> None:
        self.lista = lista
    
    def classify_list(self,strategy):
        print(f'The provided classifier is {strategy.__name__}')
        print(f'Given list {self.lista}')
        sorted_list = strategy(self.lista)
        print(f'Classified list {sorted_list}')




Let us try again all the different combinations of strategies

In [7]:
lista = [1,4,7,0,12,5,43]
reader = List_reader_functional(lista)
reader.classify_list(randomClassifier_function)
reader.classify_list(reversedClassifier_function)
reader.classify_list(sortingClassifier_function)
reader.classify_list(blackHoleClassifier_function)

The provided classifier is randomClassifier_function
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list [0, 7, 12, 5, 43, 4, 1]
The provided classifier is reversedClassifier_function
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list [43, 5, 12, 0, 7, 4, 1]
The provided classifier is sortingClassifier_function
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list [0, 1, 4, 5, 7, 12, 43]
The provided classifier is blackHoleClassifier_function
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list []


And it works just fine! (Still, a strategy pattern)

## _Common problems with these patterns described_

### I. Problem 1 - Strategies with hard coded parameters
What would happen in cases where the different strategies require hard coded parameters inside to function properly?

If that was the case, we would therefore be losing part of the interchangeability that comes with the Strategy Pattern approach.

#### Solution 1 *args and **kwargs
We already did that to some extent in the examples above, and what happens is that we are losing some of the typing information right away. Forcing us to write that info in the _.\_\_doc\_\__, etc.

### Solution 2 Creating a shared (single) Parameters class

We create a class holding the different parameters for the different strategies, that is instantiated by every different strategy uppon calling and that holds the typing information.
Let us asume a similar example as above, were we are now allowing the selection of the initial point for the new order in the list (we allow the user to select the new element in the positional index 0)

In [8]:
from dataclasses import dataclass,field
from abc import ABC, abstractmethod
import random

In [32]:
@dataclass
class StrategyParameters:
    #For the initial index classifier
    initial_index: int = 5
    #An extra parameter that allows the a classifier to remove indices
    revome_indices: list = field(default_factory=list)

#We create the abstract class for the classification method
class I_ListClassifier(ABC):
    @abstractmethod
    def classify(self,lista:list, param:StrategyParameters):
        pass

#We encapsulate an specific behaviour (algorithm or concrete behaviour) in its own class
class RandomClassifier(I_ListClassifier):
    def classify(self,lista:list,param:StrategyParameters) -> list:
        lista2 = lista.copy()
        random.shuffle(lista2)
        return lista2

class InitialNumberClassifier(I_ListClassifier):
    def classify(self,lista:list,param:StrategyParameters) -> list:
        lista2 = lista[param.initial_index:].copy()
        lista2.extend(lista[:param.initial_index])
        return lista2

class RemoveIndicesClassifier(I_ListClassifier):
    def classify(self,lista:list,param:StrategyParameters) -> list:
        lista2 = [el for i,el in enumerate(lista) if i not in param.revome_indices]
        return lista2

#We create the client that uses the classification algorithm
class List_classifier:
    def __init__(self,lista:list,strategy: I_ListClassifier) -> None:
        self.lista = lista
        self.strat = strategy
    
    def classify_list(self,initial_idx = 0, revome_indices = None):
        if type(revome_indices) != list:
            revome_indices = []
        print(f'The provided classifier is {self.strat.__class__.__name__}')
        print(f'Given list {self.lista}')
        sorted_list = self.strat().classify(lista = self.lista,param = StrategyParameters(initial_index=initial_idx,revome_indices = revome_indices))
        print(f'Classified list {sorted_list}')

In [34]:
lista = [1,4,7,0,12,5,43]
reader = List_classifier(lista,InitialNumberClassifier)
reader.classify_list(initial_idx=2)

reader = List_classifier(lista,RandomClassifier)
reader.classify_list()

reader = List_classifier(lista,RemoveIndicesClassifier)
reader.classify_list(revome_indices=[0,1,4])

The provided classifier is ABCMeta
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list [7, 0, 12, 5, 43, 1, 4]
The provided classifier is ABCMeta
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list [43, 12, 1, 7, 0, 5, 4]
The provided classifier is ABCMeta
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list [7, 0, 5, 43]


Now, the big problem is that by introducing this solution we are creating a **coupling** issue, given that the parameters class is shared by every other single class, and that the Client is now forced to know of the existance of every single parameter in that parameter class to be able to operate properly with any given new strategy.
This negates somehow the power of having a strategy pattern in place, were the idea was to **decouple** code and get multiple interchangeable functionalities with a bare minimum number of lines of code 

### Solution 3 Creaing Parameters per strategy (independent) -> Adding an initializer
To do so in an efficient way, we are going to keep using the dataclass decorator

In [35]:
#We create the abstract class for the classification method
class I_ListClassifier(ABC):
    @abstractmethod
    def classify(self,lista:list):
        pass

#We encapsulate an specific behaviour (algorithm or concrete behaviour) in its own class
class RandomClassifier(I_ListClassifier):
    def classify(self,lista:list) -> list:
        lista2 = lista.copy()
        random.shuffle(lista2)
        return lista2

@dataclass
class InitialNumberClassifier(I_ListClassifier):
    initial_index: int = 5 #Using the dataclass decorator we are avoinding to write the actual .__init__ method here
    def classify(self,lista:list) -> list:
        lista2 = lista[self.initial_index:].copy()
        lista2.extend(lista[:self.initial_index])
        return lista2

@dataclass
class RemoveIndicesClassifier(I_ListClassifier):
    revome_indices: list = field(default_factory=list)
    def classify(self,lista:list) -> list:
        lista2 = [el for i,el in enumerate(lista) if i not in self.revome_indices]
        return lista2

#We create the client that uses the classification algorithm - And now decoupled from the specifics of each algorithm
class List_classifier:
    def __init__(self,lista:list,strategy: I_ListClassifier) -> None:
        self.lista = lista
        self.strat = strategy
    
    def classify_list(self):
        print(f'The provided classifier is {self.strat.__class__.__name__}')
        print(f'Given list {self.lista}')
        sorted_list = self.strat.classify(lista = self.lista)
        print(f'Classified list {sorted_list}')

We can now initialize the different strategies independently and feed them to the Client

In [36]:
lista = [1,4,7,0,12,5,43]
reader = List_classifier(lista,InitialNumberClassifier(initial_index=2))
reader.classify_list()

reader = List_classifier(lista,RandomClassifier())
reader.classify_list()

reader = List_classifier(lista,RemoveIndicesClassifier(revome_indices=[0,1,4]))
reader.classify_list()

The provided classifier is InitialNumberClassifier
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list [7, 0, 12, 5, 43, 1, 4]
The provided classifier is RandomClassifier
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list [12, 43, 1, 5, 4, 0, 7]
The provided classifier is RemoveIndicesClassifier
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list [7, 0, 5, 43]


And this is by far the best usage of classes for the strategy pattern. We are taking advantage of the fact that a class is contains both information (parameters, data) and methods.

### I. Problem 2 - What if we want to reduce dependency in the imports?
This would be a problem presented in cases where we have several scripts containing different functionalities (encoded in classes), and we want to avoid the inconvenience of having to import every single time these classes elsewhere.

To get this, we would need **Protocols** (Python 3.8+)

In [39]:
from typing import Protocol
from dataclasses import dataclass, field

So, let's get the same classes as before describing different strategies, but with protocols

In [40]:
#We create the abstract class for the classification method
class I_ListClassifier(Protocol):
    def classify(self,lista:list):
        ...
#The 3 dots are a common way of ending protocol creations

#We encapsulate an specific behaviour (algorithm or concrete behaviour) in its own class
class RandomClassifier:
    def classify(self,lista:list) -> list:
        lista2 = lista.copy()
        random.shuffle(lista2)
        return lista2

@dataclass
class InitialNumberClassifier:
    initial_index: int = 5 #Using the dataclass decorator we are avoinding to write the actual .__init__ method here
    def classify(self,lista:list) -> list:
        lista2 = lista[self.initial_index:].copy()
        lista2.extend(lista[:self.initial_index])
        return lista2

@dataclass
class RemoveIndicesClassifier:
    revome_indices: list = field(default_factory=list)
    def classify(self,lista:list) -> list:
        lista2 = [el for i,el in enumerate(lista) if i not in self.revome_indices]
        return lista2

#We create the client that uses the classification algorithm - And now decoupled from the specifics of each algorithm
class List_classifier:
    def __init__(self,lista:list,strategy: I_ListClassifier) -> None:
        self.lista = lista
        self.strat = strategy
    
    def classify_list(self):
        print(f'The provided classifier is {self.strat.__class__.__name__}')
        print(f'Given list {self.lista}')
        sorted_list = self.strat.classify(lista = self.lista)
        print(f'Classified list {sorted_list}')

In [41]:
lista = [1,4,7,0,12,5,43]
reader = List_classifier(lista,InitialNumberClassifier(initial_index=2))
reader.classify_list()

reader = List_classifier(lista,RandomClassifier())
reader.classify_list()

reader = List_classifier(lista,RemoveIndicesClassifier(revome_indices=[0,1,4]))
reader.classify_list()

The provided classifier is InitialNumberClassifier
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list [7, 0, 12, 5, 43, 1, 4]
The provided classifier is RandomClassifier
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list [0, 12, 4, 5, 7, 1, 43]
The provided classifier is RemoveIndicesClassifier
Given list [1, 4, 7, 0, 12, 5, 43]
Classified list [7, 0, 5, 43]


Obviously, using a notebook this is completly non-relevant. We can better see the advantage of using protocols when doing scripting for a certain application (WhatEELS much? hehehe)
Also, a second added advantaga is that in some cases we can define protocols separately in different parts of the scripts for different functionalities (depending on which operation is carried out in said parts of the scripts). This avoids the creation of very convoluted ABS classes, etc.